Prajwal Srivastava

Week 3 Assignment

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

df1 = pd.read_csv('/content/train.csv')
df2 = pd.read_csv('/content/test.csv')

y = df1['SalePrice']
df1.drop('SalePrice', axis=1, inplace=True)

data = pd.concat([df1, df2], sort=False)

num = data.select_dtypes(include=['number']).columns
for col in num:
    data[col] = data[col].fillna(data[col].median())

cat = data.select_dtypes(include=['object']).columns
for col in cat:
    data[col] = data[col].fillna(data[col].mode()[0])

ord_cols = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond',
            'HeatingQC', 'KitchenQual', 'GarageQual', 'GarageCond',
            'PoolQC', 'FireplaceQu']

scale = {'Ex': 5, 'Gd': 4, 'TA': 3, 'Fa': 2, 'Po': 1, np.nan: 0}
for col in ord_cols:
    if col in data.columns:
        data[col] = data[col].map(scale)

data = pd.get_dummies(data)

data['TotalSF'] = data['TotalBsmtSF'] + data['1stFlrSF'] + data['2ndFlrSF']

X_tr = data[:len(df1)]
X_te = data[len(df1):]
y_tr = y

X_tr.to_csv("X_train_processed.csv", index=False)
X_te.to_csv("X_test_processed.csv", index=False)
y_tr.to_csv("y_train.csv", index=False)

print("Preprocessing complete!")


Preprocessing complete!


In [3]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
import numpy as np

X = pd.read_csv("X_train_processed.csv")
T = pd.read_csv("X_test_processed.csv")
y = pd.read_csv("y_train.csv")

mdl = LinearRegression()
mdl.fit(X, y)

preds = mdl.predict(X)
rmse = np.sqrt(mean_squared_error(y, preds))
print("Training RMSE:", rmse)

scores = cross_val_score(mdl, X, y.values.ravel(), scoring="neg_root_mean_squared_error", cv=10)
cv_rmse = -scores.mean()
print("Cross-Validation RMSE (10-fold):", cv_rmse)

test_out = mdl.predict(T)

print("There are", len(test_out), "house prices predicted.")
idx = int(input(f"Enter the index (0 to {len(test_out)-1}) of the house you want to see the predicted price for: "))
if 0 <= idx < len(test_out):
    print(f"Predicted price of house at index {idx} is: {test_out[idx]}")
else:
    print("Invalid index.")


Training RMSE: 22180.89274733687
Cross-Validation RMSE (10-fold): 33643.94463403646
There are 1459 house prices predicted.
Enter the index (0 to 1458) of the house you want to see the predicted price for: 2
Predicted price of house at index 2 is: [181186.93716794]
